QAOA for Max-Cut — Learning Qiskit

Here I use Qiskit to explore how the Quantum Approximate
Optimization Algorithm (QAOA) works on a Max-Cut problem

The basic idea:
- We have a set of points (nodes) connected by lines (edges)
- We want to split the points into two groups so that as many lines as possible
  go between the groups
- The number of “between-group” lines is called the cut size which is what will be maximized

Why QAOA?
- QAOA is a hybrid quantum–classical algorithm for solving optimization problems
- The quantum circuit generates solutions
- A classical loop adjusts the circuit parameters for optimization

Plan for this notebook:
1. Make a small random graph
2. Build a simple QAOA circuit for it
3. Run the circuit on Qiskit’s Aer simulator to get measurement results
4. Score the results by their cut size
5. Compare to the exact best cut for the same graph
6. Plot the best split we find

Goal: Understand the basics of building, running, and analyzing a quantum circuit in Qiskit

Reflection: This uses the same principles behind discretizing TDSE on a grid for solving 

Next logical steps: going to p=2 for another optimization layer, use scipy for param opt instead of brute force 


In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from itertools import product

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer

In [27]:
def cut_value(G, bitstring: str) -> int:
    """
    Score a bitstring ("0101") for Max-Cut
    +1 for every edge whose endpoints land in different groups (0 vs 1)
    """
    z = np.array([1 if c == '1' else 0 for c in bitstring], dtype=int) # stores as array 
    return sum(int(z[u] != z[v]) for u, v in G.edges()) # summing the nodes in each edge if different group

In [28]:
def expected_cut_from_counts(G, counts: dict) -> float:
    """
    Turn raw measurement counts into an expected cut size.
    (Weighted average of cut_value over all observed bitstrings)
    """
    shots = sum(counts.values()) # times the circut was ran 
    return sum((freq / shots) * cut_value(G, s) for s, freq in counts.items()) # probability * cut edges so it's like a weighting factor 


In [29]:
def qaoa_circuit_p1(G, gamma: float, beta: float) -> QuantumCircuit:
    """
    Build a p=1 QAOA circuit for Max-Cut (1 cost layer -> 1 mixer layer -> measure)
    - Start in uniform superposition (H on all qubits)
    - Cost layer per edge: CX(i,j) -> RZ(2γ) on j -> CX(i,j)
    - Mixer: RX(2β) on each qubit
    - Measure all qubits
    """
    # start in uniform superposition by applying Hadamard gate (putting the qubit into the sp)
    n = G.number_of_nodes() # nodes in graph
    qc = QuantumCircuit(n, n) # create qc with n qubits and n classical to store measurement 
    qc.h(range(n)) # apply H gate for equal super pos 

    # cost layer for 1 edge(i,j), adds phase for max-cut
    for i, j in G.edges():
        qc.cx(i, j) # ties bits together so z-rotation depends on i and j
        qc.rz(2 * gamma, j) # zz-rotation that adds phase based off of parity 
        qc.cx(i, j) # unties to restore structure of circuit 

    # mixer layer for phase difference interference so prob is towards better cuts 
    for q in range(n):
        qc.rx(2 * beta, q) # RX rotation by angle 2*beta on qubit q for prob to be either 0 or 1

    qc.measure(range(n), range(n)) # actually measure and store as 0 or 1

    return qc

In [ ]:
def brute_force_maxcut(G: float) -> list:
    """
    Max-cut exact calculation for comparsion to QAOA on small graph G
    """
    n = G.number_of_nodes() # nodes 
    best, best_bs = -1, None # highest cut size, corresponding bitstring

    for bits in product('01', repeat=n): # loop over every possible bitstring, repeat over n nodes 
        s = ''.join(bits) # tuple to bitstring 
        val = cut_value(G, s) # edges 
        if val > best:
            best, best_bs = val, s
    return best, best_bs

In [30]:
# setting up a small random graph
seed = 1 # for reproducibility, basically telling where the randomizer to start each time
G = nx.erdos_renyi_graph(n=5, p=0.5, seed=seed) # the graph to solve the max-cut problem on (nodes, prob, seed)

# circuit params to tune 
gamma = np.pi/4
beta  = np.pi/8

# make the circuit
qc = qaoa_circuit_p1(G, gamma, beta)
# print(qc.draw())  


In [ ]:
# running Aer 
backend = Aer.get_backend("aer_simulator")
tqc = transpile(qc, backend) # transpile circuit for sim 

shots = 1000
result = backend.run(tqc, shots=shots, seed_simulator=seed).result()
counts = result.get_counts()

# print("Counts (top few):", dict(sorted(counts.items(), key=lambda kv: kv[1], reverse=True)[:5]))
exp_cut = expected_cut_from_counts(G, counts)
# print("Expected cut (with these params):", round(exp_cut, 3))
most_probable = max(counts, key=counts.get)
# print("Most common bitstring:", most_probable, "  cut:", cut_value(G, most_probable))



In [32]:
# # --- Plot the graph structure (before QAOA) ---

# pos = nx.spring_layout(G, seed=1)  # layout for consistent positions
# plt.figure()
# nx.draw(G, pos, with_labels=True)
# plt.title("Graph G")
# plt.axis("off")
# plt.show()

# # --- Plot the best observed partition from the last run ---
# # (uses `counts` from your previous cell)
# from collections import Counter

# most_probable = max(counts, key=counts.get)  # bitstring with highest frequency
# partA = [i for i, b in enumerate(most_probable) if b == "0"]
# partB = [i for i, b in enumerate(most_probable) if b == "1"]

# plt.figure()
# nx.draw_networkx_nodes(G, pos, nodelist=partA)
# nx.draw_networkx_nodes(G, pos, nodelist=partB)
# nx.draw_networkx_labels(G, pos)
# nx.draw_networkx_edges(G, pos)
# plt.title(f"Best observed cut |S|={cut_value(G, most_probable)}  (bitstring={most_probable})")
# plt.axis("off")
# plt.show()


In [33]:
# optimize 
gammas = np.linspace(0.0, np.pi, 11)      # 11 points in [0, π]
betas  = np.linspace(0.0, np.pi/2, 11)    # 11 points in [0, π/2]

best_val = -1.0
best_gb = (None, None)
best_counts = None

for g in gammas:
    for b in betas:
        qc = qaoa_circuit_p1(G, g, b)
        tqc = transpile(qc, backend)
        result = backend.run(tqc, shots=shots, seed_simulator=1).result()
        counts = result.get_counts()
        exp_cut = expected_cut_from_counts(G, counts)
        if exp_cut > best_val:
            best_val = exp_cut
            best_gb = (g, b)
            best_counts = counts

print("Best expected cut:", round(best_val, 3))
print("Best (gamma, beta):", tuple(round(x, 4) for x in best_gb))

Best expected cut: 3.59
Best (gamma, beta): (0.3142, 1.2566)


In [37]:
pos = nx.spring_layout(G, seed=1)
best_bs = max(best_counts, key=best_counts.get)
partA = [i for i, b in enumerate(best_bs) if b == "0"]
partB = [i for i, b in enumerate(best_bs) if b == "1"]

# plt.figure()
# nx.draw_networkx_nodes(G, pos, nodelist=partA)
# nx.draw_networkx_nodes(G, pos, nodelist=partB)
# nx.draw_networkx_labels(G, pos)
# nx.draw_networkx_edges(G, pos)
# plt.title(f"Tuned best observed cut |S|={cut_value(G, best_bs)}  (bitstring={best_bs})")
# plt.axis("off")
# plt.show()

In [41]:
exact_val, exact_bs = brute_force_maxcut(G)
approx_ratio = (best_val / exact_val) if exact_val > 0 else 0.0

best_obs_bs = max(best_counts, key=best_counts.get)
best_obs_cut = cut_value(G, best_obs_bs)

print(f"Nodes: {G.number_of_nodes()}  Edges: {G.number_of_edges()}")
print(f"Exact Max-Cut: {exact_val}   (bitstring: {exact_bs})")
print(f"QAOA best expected cut:      {best_val:.3f}  (gamma, beta) = ({best_gb[0]:.4f}, {best_gb[1]:.4f})")
print(f"Approximation ratio:         {best_val/exact_val:.3f}")
print(f"Most common bitstring @best: {best_obs_bs}   cut={best_obs_cut}")


Nodes: 5  Edges: 6
Exact Max-Cut: 6   (bitstring: 01001)
QAOA best expected cut:      3.590  (gamma, beta) = (0.3142, 1.2566)
Approximation ratio:         0.598
Most common bitstring @best: 01101   cut=4
